In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

In [3]:
df = pd.read_csv('churn-modelling.csv')
df.shape

(10000, 14)

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis='columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [8]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [9]:
for column in df:
    if df[column].dtypes == 'object':
        print(f"{column}: {df[column].unique()}")

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [10]:
df = pd.get_dummies(df, columns=['Gender', 'Geography'], drop_first=True)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Male,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,False,True
2,502,42,8,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,False,True


In [11]:
scaler = MinMaxScaler()
cols_to_scale = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [12]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Male,Geography_Germany,Geography_Spain
0,0.538,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735,1,False,False,False
1,0.516,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709,0,False,False,True
2,0.304,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654,1,False,False,False
3,0.698,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120,0,False,False,False
4,1.000,0.337838,0.2,0.500246,0.000000,1.0,1.0,0.395400,0,False,False,True


In [13]:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [14]:
X = df.drop('Exited', axis='columns')
y = df['Exited']

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=37)

In [15]:
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100)

Epoch 1/100


399/399 ━━━━━━━━━━━━━━━━━━━━ 2s 925us/step - accuracy: 0.5337 - loss: 0.6884
Epoch 2/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - accuracy: 0.6577 - loss: 0.6483
Epoch 3/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step - accuracy: 0.6902 - loss: 0.5993
Epoch 4/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - accuracy: 0.7098 - loss: 0.5677
Epoch 5/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - accuracy: 0.7275 - loss: 0.5447
Epoch 6/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step - accuracy: 0.7402 - loss: 0.5278
Epoch 7/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step - accuracy: 0.7432 - loss: 0.5156
Epoch 8/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - accuracy: 0.7549 - loss: 0.5009
Epoch 9/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.7639 - loss: 0.4912
Epoch 10/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - accuracy: 0.7592 - loss: 0.4860
Epoch 11/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - accuracy: 0.7612 - loss: 0.4868
Epoch 12/100
399/399 ━━━━━━━━━━

In [16]:
model.evaluate(X_test, y_test)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 0.8001 - loss: 0.4292


[0.4277472198009491, 0.7966101765632629]

In [17]:
y_pred = np.round(model.predict(X_test))

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.80      0.80      1642
           1       0.79      0.80      0.79      1544

    accuracy                           0.80      3186
   macro avg       0.80      0.80      0.80      3186
weighted avg       0.80      0.80      0.80      3186

